In [1]:
%pip install scikit-allel pandas requests --quiet
# If running in a constrained environment, consider installing via conda:
# conda install -c conda-forge scikit-allel pandas requests

Note: you may need to restart the kernel to use updated packages.


## Part 1: VCF parser (scikit-allel)

In [2]:
# part1_vcf_parser.py

import pandas as pd
import allel
import numpy as np
import gzip
import io
import os


def parse_vcf(vcf_file, alt_number=5):
    """Parse a VCF into a pandas DataFrame using scikit-allel.

    This function tries to read the variants block from the VCF and
    expand multi-allelic records into one row per ALT allele, producing
    the same DataFrame shape as the previous implementation.
    """
    # Handle gzipped files that may not have a .gz extension by
    # checking the magic bytes. Provide a text file-like object
    # to scikit-allel so it doesn't attempt to decode compressed bytes.
    close_handle = False
    fileobj = None

    if isinstance(vcf_file, str):
        # If path exists, inspect first two bytes for gzip magic
        if os.path.exists(vcf_file):
            try:
                with open(vcf_file, 'rb') as fh:
                    magic = fh.read(2)
            except PermissionError as e:
                import getpass
                user = getpass.getuser()
                msg = (
                    f"Permission denied when opening '{vcf_file}' for reading.\n"
                    f"Current user: {user}.\n"
                    "Possible causes: file is locked, insufficient NTFS permissions, or the file is open in another program.\n"
                    "Suggested checks:\n"
                    " - Verify file exists and is readable: in PowerShell run `Test-Path .\\sample.vcf`\n"
                    " - Inspect permissions: `Get-Acl .\\sample.vcf | Format-List` or `icacls .\\sample.vcf`\n"
                    " - If needed, grant read permission: `icacls .\\sample.vcf /grant \"$env:USERNAME\":(R)` (run as admin if required).\n"
                    " - Ensure no other program is locking the file (close editors/viewers).\n"
                )
                raise PermissionError(msg) from e
            # scikit-allel requires either a file path string or binary file object,
            # not a text mode file object. Pass the file path string directly.
            fileobj = vcf_file
        else:
            # Pass through to allel which will raise a helpful error
            fileobj = vcf_file
    else:
        # Assume file-like object was passed in
        fileobj = vcf_file

    # Read as many fields as scikit-allel provides; alt_number controls ALT columns
    try:
        vcf_dict = allel.read_vcf(fileobj, fields=None, alt_number=alt_number)
    finally:
        if close_handle and hasattr(fileobj, 'close'):
            try:
                fileobj.close()
            except Exception:
                pass

    # Collect variant-level arrays returned under keys like 'variants/CHROM'
    variants = {}
    for k, v in vcf_dict.items():
        if k.startswith('variants/'):
            variants[k.split('/', 1)[1]] = v

    n = len(variants.get('CHROM', []))
    records = []

    for i in range(n):
        chrom = variants.get('CHROM')[i]
        pos = int(variants.get('POS')[i]) if 'POS' in variants else None
        ref = variants.get('REF')[i] if 'REF' in variants else None
        alts = variants.get('ALT')[i] if 'ALT' in variants else []
        qual = variants.get('QUAL')[i] if 'QUAL' in variants else None
        vid = variants.get('ID')[i] if 'ID' in variants else None
        filt = variants.get('FILTER')[i] if 'FILTER' in variants else None

        # Build INFO dictionary from keys that start with 'INFO/' in the top-level dict
        info_dict = {}
        for k, arr in vcf_dict.items():
            if k.startswith('INFO/'):
                info_key = k.split('/', 1)[1]
                try:
                    info_val = arr[i]
                except Exception:
                    info_val = None
                info_dict[info_key] = info_val

        # Normalize ALT into a list of strings
        if alts is None:
            alts_list = []
        elif isinstance(alts, (list, tuple, np.ndarray)):
            alts_list = [a.decode() if isinstance(a, bytes) else str(a) for a in alts if a is not None]
        else:
            alts_list = [str(alts)]

        for alt in alts_list:
            records.append({
                "CHROM": chrom,
                "POS": pos,
                "REF": ref,
                "ALT": alt,
                "QUAL": float(qual) if qual is not None else None,
                "ID": vid.decode() if isinstance(vid, (bytes, bytearray)) else vid,
                "FILTER": list(filt) if isinstance(filt, (list, tuple, np.ndarray)) else ([] if filt is None else [str(filt)]),
                "INFO": info_dict,
            })

    return pd.DataFrame(records)

In [3]:
# part2_filter_variants.py

def filter_variants(df, min_qual=30):
    """Filter variants by quality and keep rare ones"""
    filtered = df[df["QUAL"] >= min_qual].copy()

    # Example filter: keep only SNPs
    filtered = filtered[(filtered["REF"].str.len() == 1) & (filtered["ALT"].str.len() == 1)]

    return filtered

# Example
if __name__ == "__main__":
    from part_1 import parse_vcf
    df = parse_vcf("sample/clinvar.vcf")
    filtered_df = filter_variants(df)
    print(filtered_df.head())

Empty DataFrame
Columns: [CHROM, POS, REF, ALT, QUAL, ID, FILTER, INFO]
Index: []


## Part 3: Annotation (original file)

In [4]:
# part3_annotation.py
import requests

def annotate_clinvar(variant):
    """Fetch Clinvar annontation using variant ID (rsID if available)"""
    if variant.get("ID") is None:
        return None
    
    url = f"https://api.ncbi.nlm.nih.gov/variation/v0/beta/refsnp/{variant['ID'].replace('rs','')}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

def annotate_myvariant(hgvs_notation):
    """Annotate variant using Myvariant.info"""
    url = f"https://myvariant.info/v1/variant/{hgvs_notation}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

## Part 4: Prioritization (original file)

In [5]:
# part4_priorities.py
def prioritize_variants(df, clinvar_annotations):
    """Mark variants that are rare and pathogenic"""
    prioritized = []


    for i, row in df.iterrows():
        clinvar = clinvar_annotations.get(row["ID"], {})
        if clinvar:
            clinical_significance = clinvar.get("clinical_significance", "unknown")
            if "pathogenic" in clinical_significance.lower():
                prioritized.append((row.to_dict(), clinvar))

    return prioritized

## Part 5: Reporting (original file)

In [6]:
# part5_report.py
import pandas as pd

def save_report(priotized_variants, output_file="varquest_report.csv"):
    """Save prioritized variants into CSV"""
    records = []


    for variant, annotations in priotized_variants:
        record.append({
            "CHROM": variant["CHROM"],
            "POS": variant["POS"] ,
            "REF": variant["REF"],
            "ALT": variant["ALT"],
            "ID": variant["ID"],
            "Clinvar_Significance": annotations.get("clinical_significance", "NA")
        })

    pd.DataFrame(records).to_csv(output_file, index=False)
    print(f"Report saved as {output_file}")

## Usage Example
Run the parser on `sample.vcf` and show the first lines. Note: some of the other parts contain minor syntax or naming issues (see the Notes cell) and may need small fixes before running the full pipeline.

In [7]:
# Usage example: parse clinvar.vcf.gz
from part_1 import parse_vcf
print(df.head())

  CHROM    POS REF ALT  QUAL       ID FILTER INFO
0     1  66926  AG   A   NaN  3385321     []   {}
1     1  66926  AG       NaN  3385321     []   {}
2     1  66926  AG       NaN  3385321     []   {}
3     1  66926  AG       NaN  3385321     []   {}
4     1  66926  AG       NaN  3385321     []   {}


## Notes and Known Issues
- `part_2.py`, `part_5.py`, and `integrated.py` in the workspace contain syntax/name bugs (missing colons, wrong function names, typos).
- I intentionally included the original files as reference; if you want, I can also fix those bugs and make the full pipeline runnable.
- `scikit-allel` may require additional dependencies (NumPy, zarr) and can be installed via conda for the smoothest experience.